# Reading in the data

Here I will use the popular library "pandas" to import and store the data in a DataFrame. DataFrames are extremely powerful for exploratory data analysis like this because they allow for intricate queries to run. Once I have finished cleaning the data, I will output it to a .csv file.

In [27]:
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv('../data/the_office_lines_scripts.csv')

### Cleaning the 'speaker' data 

Every task has the speaker (IE the character speaking) at its core. Unfortunately, the speaker data from the .csv file that I have is riddled with typos and lacks a consistent format. I will take a multi-step approach to combat this.

#### Removing common punctuation

Upon examining the data, I found that several punctuating marks were very common. However, these marks are not used consistently and will cause groupings to fail as we continue through the project. In order to prevent this, I will standardize to data by removing the most common punctuating marks which are not used to delimit multiple characters

#### Correcting obvious misspellings

The spelling of character names is remarkably inconsistent in the provided script. For example, there are at least 10 different spellings of the name "Michael" -- again, this will cause groupings to fail. So, I will map the obvious ones to their correct spelling.

* Note here that some misspellings are less clear, for example- is "Helen" meant to be "Helene" or is there actually a side character named "Helen"?
* In the interest of time, I have opted to ignore any misspellings which require context and focus on the most obvious ones only


#### Split apart multiple speakers

This dataset does not split lines into a single speaker. The impact of this is that if "Michael/Dwight" say something together, then that would get filed separately from "Michael" or "Dwight". In reality, we want one speaker per row. So, I will split apart based on all the delimiters I can find in the data

#### Pull known speakers from the wiki article

Since the speaker data is both extremely messy and extremely important, we may reduce the number of errors by cross-referencing it with the known characters from the office wiki page:
https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_characters

Ultimately, on account of time constraints I will not attempt to manually correct every instance in the data where a character does not match the known character list. Instead, I will take the steps listed above and perform an exclusive join to the list of known characters, effectively erasing any lines by unknown characters

#### Cross reference with others who have done similar projects

Finally, I will search the web to find others who have performed data cleanup on this dataset and cross-reference my work with theirs to see how it compares.

#### Normalize casing + remove common punctuation

Upon examining the data, I found that several punctuating marks were very common. However, these marks are not used consistently and will cause groupings to fail as we continue through the project. In order to prevent this, I will standardize to data by removing the most common punctuating marks which are not used to delimit multiple characters.

In addition, I will normalize the casing to lower case.

In [4]:
def remove_common_punctuation(speaker: str):
    return speaker.lower().replace('.','').replace("'", '').replace(":", '')

df['speaker'] = df['speaker'].apply(remove_common_punctuation)

#### Correcting obvious misspellings

The spelling of character names is remarkably inconsistent in the provided script. For example, there are at least 10 different spellings of the name "Michael" -- again, this will cause groupings to fail. So, I will map the obvious ones to their correct spelling.

* Note here that some misspellings are less clear, for example- is "Helen" meant to be "Helene" or is there actually a side character named "Helen"?
* In the interest of time, I have opted to ignore any misspellings which require context and focus on the most obvious ones only

In [5]:
def correct_speaker_misspellings(speaker: str):
    spelling_map = {
        "chares": "charles",
        "anglea": "angela",
        "carrol": "carol",
        "carroll": "carol",
        "dacvid": "david",
        "walalace": "wallace",
        "wallcve": "wallace",
        "deagnelo": "deangelo",
        "denagelo": "deangelo",
        "dwightkschrute": "dwight",
        "michae": "michael",
        "micael": "michael",
        "micahel": "michael",
        "michaels ad": "michael",
        "michal": "michael",
        "micheal": "michael",
        "michel": "michael",
        "mihael": "michael",
        "miichael": "michael",
        "michaell": "michael",
        "todd packer": "todd",
        "phylis": "phyllis",
        "phyliss": "phyllis",
        "phylliss": "phyllis",
    }
    for i in spelling_map:
        speaker = re.sub(fr"\b{i}\b", spelling_map[i], speaker)
    return speaker
df['speaker'] = df.speaker.apply(correct_speaker_misspellings)

#### Split apart multiple speakers

This dataset does not split lines into a single speaker. The impact of this is that if "Michael/Dwight" say something together, then that would get filed separately from "Michael" or "Dwight". In reality, we want one speaker per row. So, I will split apart based on all the delimiters I can find in the data

In [6]:
def split_speakers(speaker: str):
    speaker = re.sub(r"\[.+\]", '', speaker)
    delimiter = '######'
    split_delimiters = [', and ', ',', ' and ', ' & ', '/', ' ,']
    for i in split_delimiters:
        speaker = speaker.replace(i,  delimiter).strip(' ')
    return speaker.split(delimiter)

df['speaker'] = df.speaker.apply(split_speakers)
df = df.explode('speaker').reset_index(drop=True)
df['speaker'] = df.speaker.str.strip()

#### Pull known speakers from the wiki article

Since the speaker data is both extremely messy and extremely important, we may reduce the number of errors by cross-referencing it with the known characters from the office wiki page:
https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_characters

In order to generate this list, I opted to manually copy + paste + clean the data. I could have considered a script to do this, but it would have had diminishing returns compared to a simple vim macro.

Ultimately, on account of time constraints I will not attempt to manually correct every instance in the data where a character does not match the known character list. Instead, I will take the steps listed above and perform an exclusive join to the list of known characters, effectively erasing any lines by unknown characters

Here are the characters mentioned in the wiki:

1. Michael Scott
1. Dwight Schrute
1. Jim Halpert
1. Pam Beesly
1. Ryan Howard
1. Andy Bernard
1. Robert California
1. Stanley Hudson
1. Kevin Malone
1. Meredith Palmer
1. Angela Martin
1. Oscar Martinez
1. Phyllis Vance
1. Roy Anderson
1. Jan Levinson
1. Toby Flenderson
1. Kelly Kapoor
1. Creed Bratton
1. Darryl Philbin
1. Erin Hannon
1. Gabe Lewis
1. Holly Flax
1. Nellie Bertram
1. Clark Green
1. Pete Miller
1. David Wallace
1. Deangelo Vickers
1. Jo Bennett
1. Josh Porter
1. Charles Miner
1. Ed Truck
1. Dan Gore
1. Craig
1. Troy Underbridge
1. Karen Filippelli
1. Danny Cordray
1. A.J.
1. Ben Nugent
1. Todd Packer
1. Cathy Simms
1. Hunter
1. Rolando
1. Stephanie
1. Jordan Garfield
1. Ronni
1. Lonny Collins
1. Madge Madsen
1. Glenn
1. Jerry DiCanio
1. Phillip
1. Michael
1. Matt
1. Hidetoshi Hasagawa
1. Gary Trundell
1. Val Johnson
1. Nate Nickerson
1. Gideon
1. Bruce
1. Frank
1. Louanne Kelley
1. Devon White
1. Kendall
1. Sadiq
1. Nick
1. Tony Gardner
1. Martin Nash
1. Hannah Smoterich-Barr
1. Fred Henry
1. Merv Bronte
1. Finger Lakes Guy
1. Miserly Man
1. Carol Stills 
1. Donna Newton 
1. Lucas "Luke" Cooper
1. Gerald Halpert
1. Betsy Halpert
1. Thomas Halpert 
1. Peter Halpert 
1. Katy Moore
1. Helene Beesly 
1. William Beesly
1. Penny Beesly
1. Sylvia / Meemaw
1. Alex
1. Isabel Poreba
1. Cecelia Marie "CeCe" Halpert
1. Philip Halpert
1. Mose Schrute
1. Fannie Schrute 
1. Jeb Schrute 
1. Cameron Whitman 
1. Zeke Schrute 
1. Shirley
1. Heinrich Manheim 
1. Henry Bruegger 
1. Esther Bruegger 
1. Rolf Ahl 
1. Trevor Bortmen 
1. Melvina Whitaker
1. Ira Glicksberg 
1. Gabor Csupczyk 
1. Wolf von 
1. Robert Lipton 
1. Phillip Halsted Schrute
1. Bandit (garbage, cat)
1. Rachael Martin
1. Walter Bernard, Sr.
1. Ellen Bernard
1. Walter Bernard, Jr.
1. Jessica
1. Reed
1. Irene
1. Glenn
1. Jada Philbin
1. Justine
1. Gwyneth Philbin
1. Stacey
1. Lynn
1. Gil
1. Ravi
1. Melissa Hudson 
1. Terri Hudson 
1. Cynthia
1. Robert "Bob" Vance
1. Elbert Lapin
1. Jake Palmer 
1. Sasha Flenderson 
1. Rory Flenderson 
1. Kathy Becker (never appears)
1. Art Gould 
1. Astrid Levinson
1. Kenny Anderson 
1. Lara
1. Dan
1. Mr. Flax
1. Mrs. Flax
1. Rachel Wallace 
1. Teddy Wallace 
1. Hank Tate
1. Billy Merchant
1. Leo
1. Gino
1. Brenda Matlowe
1. Vikram
1. Al Brown
1. Elizabeth
1. Fern Widgale
1. The Prince Family
1. Brandon
1. Justin Spitzer
1. Megan
1. Deborah Shoshlefski
1. Tom Witochkin
1. The Scranton Strangler (never appears)
1. Gordon
1. The documentary film crew
1. Brian

In [20]:
from known_characters import known_characters
known_character_set = set(known_characters)
df = df[df.speaker.apply(lambda x: x in known_character_set)]

#### Cross reference with others who have done similar projects

After searching the web I found someone else who did something similar. His work is available for free on github in this repo: https://github.com/brianbuie/the-office

Pulling it into a DataFrame and manually inspecting it shows that my work is pretty similar. I will also cross-reference it when performing the tasks to see whether there are any differences which might warrant close investigation.

In [25]:
from external_the_office_cleaned_lines import cleaned_lines
records = []

for i in cleaned_lines:
    season= i['season']
    episode = i['episode']
    title = i['title']
    for index,scene in enumerate(i['scenes'] + i['deleted_scenes']):
        for line in scene:
            speaker = line['character']
            line_text = line['line']
            records.append({
                "season": season,
                "episode": episode,
                "title": title,
                "scene": index + 1,
                "speaker": speaker,
                "line_text": line_text,
            })
external_df = pd.DataFrame(records)

### Save to a .csv file

In [26]:
df.to_csv('../data/cleaned_office_data.csv', index=False)
external_df.to_csv('../data/cleaned_external_office_data.csv', index=False)